In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from ChildProject.projects import ChildProject
from ChildProject.annotations import AnnotationManager

DATA_PATH = Path('/home/engaclew/neurogen')

# Read measures
human_measures = pd.read_csv(DATA_PATH / 'human_measures_chunks.csv').fillna(0)

# Read metadata
children = pd.read_csv(DATA_PATH / 'data/L3_HIPAA_LENA_cleaned/metadata/children.csv')
recordings = pd.read_csv(DATA_PATH / 'data/L3_HIPAA_LENA_cleaned/metadata/recordings.csv')
recordings_data = recordings.merge(children, on='child_id')[['group_id', 'recording_filename', 'child_sex']]
human_measures = human_measures.merge(recordings_data, how='left', on='recording_filename')

def compute_CVC(data):
    if 'can_voc_CHI' in data.columns and 'non_can_voc_CHI' in data.columns:
        data['CVC'] = data['can_voc_CHI'] + data['non_can_voc_CHI']
    return data

human_measures = compute_CVC(human_measures)

def group_data(data):
    if 'can_voc_CHI' in data.columns and 'non_can_voc_CHI' in data.columns:
        data['CVC'] = data['can_voc_CHI'] + data['non_can_voc_CHI']
    data = data.groupby('recording_filename').agg({
        '5s_CTC': np.sum,
        'voc_dur_chi': np.sum,
        'voc_dur_och': np.sum,
        'voc_dur_mal': np.sum,
        'voc_dur_fem': np.sum,
        'voc_chi': np.sum,
        'wc_adu': np.sum,
        'CVC': np.sum,
        'group_id': 'first',
        'child_id': 'first'
    }).reset_index()
    return data

df = group_data(human_measures)
duration_cols = ['voc_dur_chi', 'voc_dur_och', 'voc_dur_mal', 'voc_dur_fem']
for col in duration_cols:
    df[col] = df[col] / (1000 * 60)  # Convert from milliseconds to minutes

/tmp/ipykernel_16992/3987423873.py:28: FutureWarning: The provided callable <function sum at 0x7f3d3959add0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data = data.groupby('recording_filename').agg({
/tmp/ipykernel_16992/3987423873.py:28: FutureWarning: The provided callable <function sum at 0x7f3d3959add0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data = data.groupby('recording_filename').agg({


In [2]:
human_measures.drop_duplicates('child_id').groupby('group_id')['child_sex'].apply(lambda x: (x == 'f').sum())

group_id
angelman_syndrome     5
autism_sibling        2
down_syndrome         2
fragile_x_syndrome    3
low_risk              2
Name: child_sex, dtype: int64

,recording_filename,segment_onset,segment_offset,child_id,duration_eaf/an1,voc_dur_chi,voc_dur_mal,voc_dur_fem,voc_dur_och,voc_chi,...,wc_mal,wc_adu,5s_CTC,non_can_voc_CHI,can_voc_CHI,speechlike_pitch,nonspeechlike_pitch,group_id,child_sex,CVC
0,20180808_111213_024881.wav,1639000,1759000,2761,120000,15070.0,0.0,0.0,0.0,8,...,0,0,0,5,3,"[247.84825319432744, 251.1031191922398, 229.59...",[],low_risk,m,8
15,20180809_181624_025481.wav,1639000,1759000,2781,120000,36692.0,0.0,33060.0,1522.0,40,...,0,136,35,36,3,"[418.60197371783755, 440.56156986801494, 379.7...",[429.6569717115748],low_risk,m,39
30,20180906_131431_024882.wav,1639000,1759000,3591,120000,1983.0,0.0,39914.0,33877.0,3,...,0,280,5,2,1,"[295.15866376379813, 270.6225060791767, 273.01...",[],down_syndrome,m,3
45,20181218_132113_024879.wav,1639000,1759000,3131,120000,13369.0,10332.0,30003.0,21264.0,23,...,31,167,26,19,4,"[331.62225996561983, 293.32250539622754, 306.8...",[],angelman_syndrome,m,23
60,20190307_091344_022872.wav,1639000,1759000,3951,120000,5651.0,0.0,1308.0,6542.0,8,...,0,9,0,6,2,"[343.8137701073527, 378.9209649634173, 339.109...",[],down_syndrome,f,8
75,20190618_135122_024883.wav,1639000,1759000,5011,120000,10263.0,5170.0,13676.0,0.0,13,...,14,50,8,9,4,"[282.93940125421204, 287.0312136166218, 288.93...",[],low_risk,m,13
90,20190828_145547_024884_2.wav,2072000,2192000,5061,120000,8006.0,0.0,66353.0,0.0,11,...,0,302,18,10,1,"[277.9320392356493, 491.6403986655598, 280.711...",[],angelman_syndrome,f,11
99,20201104_094551_024882_1.wav,1639000,1759000,3871,120000,14781.0,0.0,36294.0,0.0,27,...,0,99,40,25,2,"[277.43953555200005, 329.31633640733867, 420.9...",[],down_syndrome,m,27
114,20210127_103244_024879_1.wav,1639000,1759000,5461,120000,4422.0,0.0,256.0,0.0,8,...,0,1,1,8,0,"[312.3507776085124, 338.3181784522583, 249.223...",[],angelman_syndrome,m,8
129,20210318_114509_043298.wav,1639000,1759000,5961,120000,1358.0,0.0,22261.0,32548.0,2,...,0,134,3,2,0,"[377.06353436294785, 399.597950671351]",[],angelman_syndrome,f,2


In [2]:
import pandas as pd
import numpy as np

# Calculate total duration for each recording and get statistics
df['total_duration'] = df[duration_cols].sum(axis=1)

metrics = {
    'Cumulated speech/vocalization duration (mn)': df['total_duration'],
    "Key child's vocalizations (%)": (df['voc_dur_chi'] / df['total_duration']) * 100,
    "Other children's vocalizations (%)": (df['voc_dur_och'] / df['total_duration']) * 100,
    'Adult female speech (%)': (df['voc_dur_fem'] / df['total_duration']) * 100,
    'Adult male speech (%)': (df['voc_dur_mal'] / df['total_duration']) * 100,
    'Conversational Turn Count': df['5s_CTC'],
    'Adult Word Count': df['wc_adu'],
    'Child Vocalization Count': df['CVC']
}

# Calculate mean, min, and max for each metric
summary = pd.DataFrame({
    'Mean': {k: v.mean() for k, v in metrics.items()},
    'Min': {k: v.min() for k, v in metrics.items()},
    'Max': {k: v.max() for k, v in metrics.items()}
})

summary = summary.round(1)
print(summary)

for index, row in summary.iterrows():
    #print(index)
    print(' '.join(map(str, row.values)))

                                              Mean    Min     Max
Cumulated speech/vocalization duration (mn)    6.1    2.0    10.2
Key child's vocalizations (%)                 31.9    5.8    60.8
Other children's vocalizations (%)            14.1    0.0    39.1
Adult female speech (%)                       41.6   10.9    75.4
Adult male speech (%)                         12.4    0.0    39.5
Conversational Turn Count                     90.9   10.0   201.0
Adult Word Count                             781.0  142.0  1412.0
Child Vocalization Count                     130.7   31.0   264.0
6.1 2.0 10.2
31.9 5.8 60.8
14.1 0.0 39.1
41.6 10.9 75.4
12.4 0.0 39.5
90.9 10.0 201.0
781.0 142.0 1412.0
130.7 31.0 264.0


In [3]:
# Group by group_id and calculate statistics for each group
group_stats = {}
for group in df['group_id'].unique():
    group_data = df[df['group_id'] == group]
    
    group_metrics = {
        'Cumulated speech/vocalization duration (mn)': group_data['total_duration'],
        "Key child's vocalizations (%)": (group_data['voc_dur_chi'] / group_data['total_duration']) * 100,
        "Other children's vocalizations (%)": (group_data['voc_dur_och'] / group_data['total_duration']) * 100,
        'Adult female speech (%)': (group_data['voc_dur_fem'] / group_data['total_duration']) * 100,
        'Adult male speech (%)': (group_data['voc_dur_mal'] / group_data['total_duration']) * 100,
        'Conversational Turn Count': group_data['5s_CTC'],
        'Adult Word Count': group_data['wc_adu'],
        'Child Vocalization Count': group_data['CVC']
    }
    
    group_summary = pd.DataFrame({
        'Mean': {k: v.mean() for k, v in group_metrics.items()},
        'Min': {k: v.min() for k, v in group_metrics.items()},
        'Max': {k: v.max() for k, v in group_metrics.items()}
    })
    
    group_stats[group] = group_summary

# Print statistics for each group
for group, stats in group_stats.items():
    print(f"\nGroup: {group}")
    print("-" * 50)
    print(stats.round(1))


Group: low_risk
--------------------------------------------------
                                              Mean    Min     Max
Cumulated speech/vocalization duration (mn)    6.0    3.5    10.2
Key child's vocalizations (%)                 43.2   31.4    50.6
Other children's vocalizations (%)            16.4    0.0    39.1
Adult female speech (%)                       37.0   10.9    61.6
Adult male speech (%)                          3.4    0.0     9.0
Conversational Turn Count                     91.4   13.0   201.0
Adult Word Count                             674.2  309.0  1139.0
Child Vocalization Count                     140.2   46.0   264.0

Group: down_syndrome
--------------------------------------------------
                                              Mean    Min     Max
Cumulated speech/vocalization duration (mn)    7.3    6.6     8.7
Key child's vocalizations (%)                 24.7   10.3    42.1
Other children's vocalizations (%)            26.7   18.4    36.6
A

In [21]:
# Define the group order we want
group_order = ['low_risk', 'angelman_syndrome', 'down_syndrome', 'fragile_x_syndrome', 'autism_sibling']

# Create a MultiIndex DataFrame to store all statistics
all_stats = pd.DataFrame()

for group in group_order:
    group_data = df[df['group_id'] == group]
    
    group_metrics = {
        'Cumulated speech/vocalization duration (mn)': group_data['total_duration'],
        "Key child's vocalizations (%)": (group_data['voc_dur_chi'] / group_data['total_duration']) * 100,
        "Other children's vocalizations (%)": (group_data['voc_dur_och'] / group_data['total_duration']) * 100,
        'Adult female speech (%)': (group_data['voc_dur_fem'] / group_data['total_duration']) * 100,
        'Adult male speech (%)': (group_data['voc_dur_mal'] / group_data['total_duration']) * 100,
        'Conversational Turn Count': group_data['5s_CTC'],
        'Adult Word Count': group_data['wc_adu'],
        'Child Vocalization Count': group_data['CVC']
    }
    
    group_summary = pd.DataFrame({
        f'{group}_Mean': {k: v.mean() for k, v in group_metrics.items()},
        f'{group}_Min': {k: v.min() for k, v in group_metrics.items()},
        f'{group}_Max': {k: v.max() for k, v in group_metrics.items()}
    })
    
    all_stats = pd.concat([all_stats, group_summary], axis=1)

# Round all values to 1 decimal place
all_stats = all_stats.round(1)

# Print space-separated values
for index, row in all_stats.iterrows():
    #print(index)
    print(' '.join(map(str, row.values)))

6.0 3.5 10.2 5.6 2.0 8.2 7.3 6.6 8.7 5.5 3.7 7.8 6.8 4.7 8.7
43.2 31.4 50.6 23.7 5.8 41.6 24.7 10.3 42.1 33.7 18.5 60.8 40.7 36.6 46.2
16.4 0.0 39.1 17.0 0.0 36.4 26.7 18.4 36.6 0.9 0.0 3.4 8.9 4.0 11.8
37.0 10.9 61.6 46.9 20.5 70.2 32.1 14.0 53.4 52.4 26.3 75.4 31.3 23.1 42.0
3.4 0.0 9.0 12.4 0.9 24.6 16.5 2.0 36.7 13.0 0.0 39.5 19.0 13.9 25.7
91.4 13.0 201.0 71.4 10.0 201.0 102.2 65.0 190.0 90.2 45.0 169.0 118.0 60.0 187.0
674.2 309.0 1139.0 784.4 142.0 1412.0 998.4 659.0 1252.0 711.2 332.0 1284.0 728.0 519.0 852.0
140.2 46.0 264.0 91.1 31.0 157.0 159.6 59.0 248.0 120.0 39.0 234.0 180.7 103.0 261.0


In [5]:
human_measures.columns

Index(['recording_filename', 'segment_onset', 'segment_offset', 'child_id',
       'duration_eaf/an1', 'voc_dur_chi', 'voc_dur_mal', 'voc_dur_fem',
       'voc_dur_och', 'voc_chi', 'avg_voc_dur_chi', 'wc_fem', 'wc_mal',
       'wc_adu', '5s_CTC', 'non_can_voc_CHI', 'can_voc_CHI',
       'speechlike_pitch', 'nonspeechlike_pitch', 'group_id', 'CVC'],
      dtype='object')